# Matrix Generation

We need to generate matrices that are irreducible and aperiodic.

In [62]:
import math
import random
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy.sparse import dok_matrix

random.seed(4)

In [63]:
def is_ergodic(matrix: dok_matrix, n: int) -> bool:
    """
    An ergodic matrix is aperiodic and irreducible. By Wielandt's theorem if when the matrix is multiplied by itself m
    times, where m = (n - 1) * (n - 1) + 1, and all its entries are positive then the matrix is ergodic. n is the number
    of sates.

    :param matrix: matrix to check
    :return: true if the matrix is ergodic, false otherwise
    """
    matrix = matrix.tocsr(copy=True)
    m = (n - 1) * (n - 1) + 1
    multiplicities = [matrix]
    for i in range(int(math.log(m, 2))):
        matrix = matrix.dot(matrix)
        multiplicities.append(matrix)
    index = len(multiplicities) - 1
    res = None
    while m > 0:
        if m & 1:
            if res is None:
                res = multiplicities[index]
            else:
                res = res.dot(multiplicities[index])
        index -= 1
        m = m >> 1
    return res.count_nonzero() == n * n

In [64]:
print(is_ergodic(dok_matrix([[0.7, 0.3, 0], [0, 0, 1], [0, 0.6, 0.4]]), 3))     # False
print(is_ergodic(dok_matrix([[0.7, 0.3, 0], [0, 0.5, 0.5], [0.4, 0.6, 0]]), 3)) # True
print(is_ergodic(dok_matrix([[0.7, 0.3, 0], [0, 0, 1], [0.4, 0.6, 0]]), 3))     # True
print(is_ergodic(dok_matrix([[0, 1, 0], [0, 0, 1], [1, 0, 0]]), 3))             # False
print(is_ergodic(dok_matrix([[0, 1, 0], [0.5, 0, 0.5], [0, 1, 0]]), 3))         # False
print(is_ergodic(dok_matrix([[0, 0.5, 0.5], [0, 0.5, 0.5], [1, 0, 0]]), 3))     # True

False
True
True
False
False
True


In [65]:
# def generate(n: int, sizes: list[int], p1: float, p2: float) -> list[list[list[int]]]:
#     """
#     Generate matrices with random values in them. The values are between 0 and 1.
#
#     :param n: how many of each matrix size should be generated
#     :param sizes: the sizes of the matrices that should be generated
#     :return: the generated matrices
#     """
#     all = []
#     increase = (p2 - p1) / n
#     for size in tqdm(sizes):
#         i = 0
#         matrices = []
#         zeros_p = p1
#         while i < n:
#             matrix = dok_matrix((size, size), dtype=np.float64)
#             for j in range(size):
#                 s = 0
#                 while s == 0:
#                     row = random.choices(range(1, 101), k=size)
#                     zeros = random.sample(range(0, size), int(size * zeros_p))
#                     for index in zeros:
#                         row[index] = 0
#                     s = sum(row)
#                 row = [(x / s) for x in row]
#                 matrix[j] = row
#             if not is_ergodic(matrix, size):
#                 continue
#             zeros_p += increase
#             matrices.append(matrix)
#             all.append(matrix)
#             i += 1
#         with open(f"./data/generated/matrices_{size}_{n}.pickle", 'wb') as f:
#             pickle.dump(matrices, f)
#     return all

In [66]:
def generate(n: int, sizes: list[int]) -> list[list[list[int]]]:
    """
    Generate matrices with random values in them. The values are between 0 and 1.

    :param n: how many of each matrix size should be generated
    :param sizes: the sizes of the matrices that should be generated
    :return: the generated matrices
    """
    all = []
    for size in tqdm(sizes):
        i = 0
        matrices = []
        transitions_on_row = 2
        while i < n:
            matrix = dok_matrix((size, size), dtype=np.float64)
            indices = np.random.choice(size, size=size, replace=False)
            last = 0
            for index in indices:
                matrix[last, index] = 1
                last = index
            for _ in range(min(transitions_on_row - 1, size)):
                indices = np.random.choice(size, size=size, replace=False)
                for (row, col) in enumerate(indices):
                    matrix[row, col] = 1
            if not is_ergodic(matrix, size):
                continue
            matrices.append(matrix)
            all.append(matrix)
            i += 1
        with open(f"./data/generated2/matrices_{size}_{n}.pickle", 'wb') as f:
            pickle.dump(matrices, f)
    return all

In [67]:
matrices = generate(10, range(10, 200))

100%|██████████| 190/190 [02:29<00:00,  1.27it/s]
